In [ ]:
import openai
import os
import subprocess
import tempfile

# Set up your OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')

def programmer_agent(user_query):
    system_message = "You are a programmer. Write Python code to solve the user's problem. Provide only the code, do not include explanations."
    user_message = f"Problem:\n\n{user_query}\n\nEncapsulate your solution in functions or classes as appropriate."
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ],
        max_tokens=1500,
        temperature=0,
    )
    code = response['choices'][0]['message']['content'].strip()
    return code

def test_designer_agent(user_query):
    system_message = "You are a test designer. Write relevant Python unit tests to verify the correctness of code solving the user's problem. Use the unittest framework in Python. Provide only the test code, do not include explanations."
    user_message = f"Problem:\n\n{user_query}\n\nAssume the programmer's code is in a file named 'code.py'. Import the necessary functions or classes from 'code.py' to perform the tests."
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ],
        max_tokens=1500,
        temperature=0,
    )
    test_code = response['choices'][0]['message']['content'].strip()
    return test_code

def code_executor(code, test_code):
    with tempfile.TemporaryDirectory() as tempdir:
        # Write the programmer's code to code.py
        code_file = os.path.join(tempdir, 'code.py')
        with open(code_file, 'w') as f:
            f.write(code)

        # Write the test code to test_code.py
        test_file = os.path.join(tempdir, 'test_code.py')
        with open(f"{tempdir}/test_code.py", 'w') as f:
            f.write(test_code)

        # Run the tests using unittest
        try:
            result = subprocess.run(
                ['python', '-m', 'unittest', 'test_code.py'],
                cwd=tempdir,
                capture_output=True,
                text=True,
                check=True
            )
            output = result.stdout
            errors = result.stderr
            return True, output, errors
        except subprocess.CalledProcessError as e:
            # Tests failed
            output = e.stdout
            errors = e.stderr
            return False, output, errors

def test_agent(user_query, code, test_output, test_errors):
    system_message = "You are a test agent. Evaluate the programmer's code based on the test results and provide feedback."
    user_message = f"""
The programmer wrote the following code:

{code}

The tests produced the following output:

{test_output}

There were errors:

{test_errors}

Provide an explanation of the errors and suggest how to fix them.
"""
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ],
        max_tokens=500,
        temperature=0,
    )
    explanation = response['choices'][0]['message']['content'].strip()
    return explanation

def main():
    user_query = input("Enter your query: ")

    # Programmer writes code
    code = programmer_agent(user_query)
    print("\nProgrammer's code:")
    print(code)

    # Test designer writes tests
    test_code = test_designer_agent(user_query)
    print("\nTest Designer's test code:")
    print(test_code)

    # Execute code and tests
    success, output, errors = code_executor(code, test_code)

    if success:
        print("\nAll tests passed!")
    else:
        print("\nTests failed.")
        print("Test Output:")
        print(output)
        print("Test Errors:")
        print(errors)

        # Test agent evaluates the code
        explanation = test_agent(user_query, code, output, errors)
        print("\nTest Agent's feedback:")
        print(explanation)

if __name__ == "__main__":
    main()
